<a href="https://colab.research.google.com/github/ElenaShargina/patterns/blob/master/%D0%9F%D0%BE%D0%B2%D0%B5%D0%B4%D0%B5%D0%BD%D1%87%D0%B5%D1%81%D0%BA%D0%B8%D0%B5%20%D0%BF%D0%B0%D1%82%D1%82%D0%B5%D1%80%D0%BD%D1%8B/State.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# State / Состояние
Позволяет объекту варьировать свое поведение в зависимости от внутреннего состояния. Извне создается впечатление, что изменился класс объекта.
Используется:
- когда поведение объекта зависит от его состояния и должно изменяться во время выполнения.
- когда в коде операций встречаются состоящие из многих ветвей условные операторы, в которых выбор ветви зависит от состояния. Обычно в таком случае состояние представлено перечисляемыми константами. Часто одна и та же структура условного оператора повторяется в нескольких операциях. Паттерн "состояние" предлагает поместить каждую ветвь в отдельный класс. это позволяет трактовать состояние объекта как самостоятельный объект, который может изменяться независимо от других.

## Пример реализации 1
<img src='http://feana.ru/wp-content/uploads/2023/05/state.png'>

In [9]:
# класс контекста, через который работают клиенты.
class Connection:
    def __init__(self,port):
        self._state = TCPClosed(port)
        
    # метод, где Connection передает управление классу состояния TCPState
    def open(self):
        print('Open the connection...')
        self._state.open(self)

    def close(self):
        print('Close the connection...')
        self._state.close(self)

    def change_state(self,state):
        self._state = state

    def display(self):
        self._state.display()

# класс-состояние
class TCPState:
    def __init__(self,port):
        self._port = port
    def open(self,connection):
        pass
    def close(self,connection):
        pass
    def display(self):
        pass

# подкласс - конкретное состояние
class TCPEstablished(TCPState):
    def open(self,connection):
        pass
    
    # метод, где TCPEstablished принимает решение о смене состояния
    def close(self,connection):
        connection.change_state(TCPClosed(self._port))

    def display(self):
        print(f'Connection is in established state at {self._port} port')

# подкласс - конкретное состояние
class TCPClosed(TCPState):
    # метод, где TCPClosed принимает решение о смене состояния
    def open(self,connection):
        connection.change_state(TCPEstablished(self._port))

    def close(self,connection):
        pass
    
    def display(self):
        print(f'Connection is in closed state at {self._port} port')


print('Создадим соединение через порт 123. По умолчанию состояние канала "closed"')
connect = Connection(port=123)
connect.display()
print('Откроем соединение. Теперь состояние канала "established"')
connect.open()
connect.display()
print('Закроем соединение. Теперь состояние канала "closed"')
connect.close()
connect.display()




Создадим соединение через порт 123. По умолчанию состояние канала "closed"
Connection is in closed state at 123 port
Откроем соединение. Теперь состояние канала "established"
Open the connection...
Connection is in established state at 123 port
Закроем соединение. Теперь состояние канала "closed"
Close the connection...
Connection is in closed state at 123 port


## Пример реализации 2

В этом примере решение о смене состояния будет принимать класс контекста.

In [10]:
# класс контекста, через который работают клиенты
class Connection:
    def __init__(self,port):
        self._port = port
        self._state = ClosedState(self._port)

    # метод, где Connection сам принимает решение о смене подкласса состояния
    def open(self):
        self._state.__class__ = OpenedState

    # метод, где Connection сам принимает решение о смене подкласса состояния
    def close(self):
        self._state.__class__ = ClosedState

    def send(self,message):
        self._state.send(message)

    def display(self):
        self._state.display()

# класс-состояние
class State:
    def __init__(self,port):
        self._port = port
        
    def open(self):
        pass

    def close(self):
        pass

    def display(self):
        print(self.__class__.__name__,end=' -> ')
        self._display()

    def send(self,message):
        pass

# подкласс-состояние "закрыто"
class ClosedState(State):
    def open(self):
        print('The state is Closed.')
    def close(self):
        pass
    def _display(self):
        print(f'The state is closed at {self._port} port.')
    def send(self,message):
        print('Cant send message to closed state')

# подкласс-состояние "открыто"
class OpenedState(State):
    def open(self):
        pass
    def close(self):
        pass
    def _display(self):
        print(f'The state is opened at {self._port} port.')
    def send(self,message):
        print(f'Message {message} is successfully sent.')

print('Создадим соединение через порт 123. По умолчанию состояние канала "closed"')
context = Connection(port=123)
context.display()
print('Откроем соединение. Теперь состояние канала "open"')
context.open()
context.display()
print('Отправим сообщение.')
context.send('Hello')
print('Закроем соединение. Теперь состояние канала "closed"')
context.close()
context.display()
print('Попробуем отправить сообщение в закрытый канал.')
context.send('Hi')

Создадим соединение через порт 123. По умолчанию состояние канала "closed"
ClosedState -> The state is closed at 123 port.
Откроем соединение. Теперь состояние канала "open"
OpenedState -> The state is opened at 123 port.
Отправим сообщение.
Message Hello is successfully sent.
Закроем соединение. Теперь состояние канала "closed"
ClosedState -> The state is closed at 123 port.
Попробуем отправить сообщение в закрытый канал.
Cant send message to closed state
